# π-systems of hydrocarbons

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## The graphene layer

In [ ]:
# First, define a graphene layer (from the MoleculesInPython.ipynb notebook)
a = 1.42
n0 = 3
n1 = 3

xyz = np.array([[0, 0, 0], [a, 0, 0]])
s3 = 3**0.5
cell = a / 2 * np.array([[3, -3**0.5, 0], [3, 3**0.5, 0], [0, 0, 10]])

def tile(xyz, cell, ntile, axis):
    xyz = xyz[None, ...].copy()
    xyz = xyz + cell[None, (axis,), :] * np.arange(ntile)[:, None, None]
    cell = cell.copy()
    cell[axis, :] *= ntile
    return xyz.reshape(-1, 3), cell

txyz, tcell = tile(xyz, cell, n0, 0)
txyz, tcell = tile(txyz, tcell, n1, 1)
gra = txyz[:, :2]  # just to work in 2D
cgra = tcell[:2, :2]

In [ ]:
# Make a plot like the one in the Mathematica notebook
f, ax = plt.subplots(figsize=(9,4))
# First, the atoms...
colors = ["red", "#5555FF"]*(n0*n1)  # In our case, even/odd are different sublattices
ax.scatter(gra[:, 0], gra[:, 1], s=30**2, c=colors)
# Then the indices...
for i, loc in enumerate(gra):
    ax.text(loc[0], loc[1], str(i), verticalalignment='center', horizontalalignment="center",
            fontdict={"weight": "bold", "size": "x-large"})
# Now the lines...
def plotneighborlines(xyz, ax, neighbors=None):
    reln = a * np.array([[2, 0], [-1, 3**0.5], [-1, -3**0.5]])/2
    for i, loc in enumerate(xyz):
        if neighbors is None:
            dests = (-1)**i * reln + loc[None, :]
        else:
            dests = xyz[np.flatnonzero(neighbors[i])]
        for dest in dests:
            x, y = np.vstack((loc, dest)).T
            ax.plot(x, y, "k", linewidth=0.5)
plotneighborlines(gra, ax)
ax.set_aspect('equal', adjustable='box')

## Function: Create hamiltonian for coordinates

In [ ]:
def hamiltonian(xyz):
    dist = np.linalg.norm(xyz[None, :, :] - xyz[:, None, :], axis=2)
    return np.where((dist < (a + 0.1)) & (dist > 0.1), -1, 0)
# Output a test
print(hamiltonian(gra[[1, 2, 3, 8, 7, 6]]))

## Function: Plot the eigenstates of a hamiltonian
Where the color is according to the phase and size according to the norm. Instead of blue/red we use a cyclic colormap (hsv, twilight, twilight_shifted exist)

In [ ]:
def showstates(xyz):
    H = hamiltonian(xyz)
    es, vs = np.linalg.eigh(H)
    f, axes = plt.subplots(1, len(xyz), figsize=(9.5, 2))
    for e, v, ax in zip(es, vs.T, axes):
        plotneighborlines(xyz, ax, neighbors=H)
        phase = np.angle(v)
        v = np.abs(v)**2 * 30**2
        ax.scatter(xyz[:, 0], xyz[:, 1], s=v, c=phase, cmap="hsv", vmin=-np.pi, vmax=np.pi)
        ax.set_aspect('equal', adjustable='box')
        ax.set_title(f"E={e:.3f}")
    ne = len(H) / 2
    nei = int(ne)
    if ne != nei:
        # In this case homo==lumo due to spin degeneracy
        return 0
    gap = es[nei] - es[nei-1]
    return gap

gap = showstates(gra[[1, 2, 3, 8, 7, 6]])
print(f"HOMO-LUMO gap: {gap:.3f}")

## Function: Plot the spectrum of a hamiltonian

In [ ]:
def showspectrum(xyz):
    H = hamiltonian(xyz)
    es, vs = np.linalg.eigh(H)
    f, ax = plt.subplots()
    eu, neu = np.unique(es.round(1), return_counts=True)
    ax.bar(eu, height=neu, width=0.1)
    ax.set_ylabel("degeneracy")
    ax.set_xlabel("energy")
    ax.grid(axis="y")
showspectrum(gra[[1, 2, 3, 8, 7, 6]])

## Exercise : 
Try to understand the code above. Which are the HOMO and LUMO levels? What is special about the highest/lowest energy state? Which states seems to break the symmetry of the system -- how is that possible? 

## Exercise :
Consider the same for Coronene (see Fig. 5 in the notes)

## Exercise: 
Consider also Anthracene 

## Exercise: 
Is it always so that the spectrum is symmetric around E=0? Explore -- what is the relation between the state at E and -E?

## What happens to the Hamiltonian if you replace the basis - set with: 
`[[1, 0, 0 ..], [0, -1, 0, 0 ..], [0, 0, 1, 0 ...], [0, 0, 0, -1] ..]` (the $\pi$-orbital on every 2nd atom is multiplied by minus one).

# Extras:

## Take a look at the Notebook "MoleculesInPython", and 
Make your code able to set up the $\pi$-electron tight-binding hamiltonian for a molecule defined from a list of C atoms either from a xyz file read-in or somewhere else and solve for the eigenvalues and eigenstates.

## Make plots (3 D) of the eigenstates (generalize the 2D plots)
Hint
```python
from mpl_toolkits.mplot3d import Axes3D
f, ax = plt.subplots(subplot_kw=dict(projection="3d"))
ax.scatter(*xyz.T)
```


## Try out your code on a C60 molecule
Would require the 3D-generalization!

## Use the ASE for plotting 3D atomic structure

https://wiki.fysik.dtu.dk/ase

Installation: https://wiki.fysik.dtu.dk/ase/install.html

In [ ]:
from ase import Atoms
from ase.visualize import view # external viewer
from ase.visualize.plot import plot_atoms # view inside jupyter

In [ ]:
molecule = Atoms(['C' for i in txyz],
             positions=txyz,
             cell=cell,
             pbc=[1, 1, 0])

In [ ]:
view(molecule)  # open external viewer

In [ ]:
f, a = plt.subplots(); plot_atoms(molecule, a, rotation="0x,0y,0z") # view inside jupyter

If you want to know something about something you can put a "?" in front:

In [ ]:
?plot_atoms